# Automatic Peak Finder

## Lightnovo ApS

### info@lightnovo.com

Copyright 2024 Lightnovo ApS

Licensing: MIT license

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import os

import pybaselines
from scipy.signal import find_peaks

import datetime

In [ ]:
sns.set(font_scale=1.5)
sns.set_style('whitegrid')

In [ ]:
plt.rcParams.update({'figure.max_open_warning': 0})

In [ ]:
#Raman spectrum class
class raman_spectrum:
    def __init__(self,fname):
        self.fname = fname
        self.fname0 = self.fname.split('/')[-1]
        self.df = pd.read_csv(fname + '.tsv', sep='\t', header =7)
        self.df.rename(columns={'Tags':'Raman Shift, cm-1', self.df.columns[1]:'Intensity, %'}, inplace = True)
        self.df.replace([np.inf, -np.inf], np.nan, inplace=True)
        self.df.dropna(inplace = True)
        self.baseline()
        self.normalize()
   
  
    def baseline(self):
        pass
        self.df["Baseline, %"] = pybaselines.whittaker.arpls(self.df["Intensity, %"])[0]
        self.df["Baseline Corrected, %"] = self.df["Intensity, %"] - self.df["Baseline, %"] 
        
    def normalize(self):
        self.df["Normalized Intensity"] = (self.df["Intensity, %"] - ( min0:=self.df["Intensity, %"].min()) )/(self.df["Intensity, %"].max() - min0) 
        self.df["Normalized Baseline Corrected"] = (self.df["Baseline Corrected, %"] - ( min0:=self.df["Baseline Corrected, %"].min()) )/(self.df["Baseline Corrected, %"].max() - min0) 
    
    def plot(self, color, label):
        sns.lineplot(self.df, x = "Raman Shift, cm-1", y = "Intensity, %", color = color, label = label)
        plt.xlabel(r'Raman Shift, $cm^{-1}$') 
    
    def plot(self, color, label):
        sns.lineplot(self.df, x = "Raman Shift, cm-1", y = "Intensity, %", color = color, label = label)
        plt.xlabel(r'Raman Shift, $cm^{-1}$') 
  
    def plot_baseline(self, color, label):
        sns.lineplot(self.df, x = "Raman Shift, cm-1", y = "Baseline, %", color = color, label = label)
        plt.xlabel(r'Raman Shift, $cm^{-1}$') 
        
    def plot_baseline_corrected(self, color, label):
        sns.lineplot(self.df, x = "Raman Shift, cm-1", y = "Baseline Corrected, %", color = color, label = label)
        plt.xlabel(r'Raman Shift, $cm^{-1}$') 
        
    def plot_baseline_corrected_n(self, color, label):
        sns.lineplot(self.df, x = "Raman Shift, cm-1", y = "Normalized Baseline Corrected", color = color, label = label)
        plt.xlabel(r'Raman Shift, $cm^{-1}$') 
        
    def plot_n(self, color, label, marker = None):
        sns.lineplot(self.df, x = "Raman Shift, cm-1", y = "Normalized Intensity", marker = marker, color = color, label = label)
        plt.xlabel(r'Raman Shift, $cm^{-1}$') 

In [ ]:
tic = datetime.datetime.now()

In [ ]:
dir_path = "Data"

In [ ]:
notebook_dir = os.getcwd()
os.chdir(dir_path)
files = os.listdir()
files.sort(key=lambda x: os.path.getmtime(x))

In [ ]:
print(files)

In [ ]:
files = [f.split('.')[0] for f in files]

In [ ]:
print(files)

In [ ]:
spectra = []

for f in files:
    spectra.append(raman_spectrum(f))

In [ ]:
os.chdir(notebook_dir)

In [ ]:
for s in spectra:
    
    
    df = s.df[s.df["Raman Shift, cm-1"].between(500,2000)] # specify range
    
    peaks, _ = find_peaks(df["Normalized Baseline Corrected"], height = 0.05, distance = 10) # specify min height and distance between peaks
    
    plt.figure(figsize = (10,5))
    s.plot_baseline_corrected_n(color = 'red',label = s.fname0)
    sns.scatterplot( df.iloc[peaks], x = "Raman Shift, cm-1", y = "Normalized Baseline Corrected", marker = 'x', color = 'black', s = 100);
    plt.savefig(s.fname0 + '.png', dpi = 300, bbox_inches='tight')

    print(s.fname0 + " peaks (cm-1):")
    
    for p in peaks:
        print(df.iloc[p]["Raman Shift, cm-1"])

In [ ]:
toc = datetime.datetime.now()
print(toc-tic)